# Cambiamenti

- Provare ad usare un solo hidden layer
- Provare a cambiare funzione d'attivazione dell'output layer da lineare a ReLu
- Cambiare l'ultimo grafico


In [1]:
import numpy as np
import pandas as pd
from re import sub
from time import time

from comet_ml import Experiment

from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout
from keras_tqdm import TQDMNotebookCallback as ktqdm
from keras.utils import normalize
from keras.wrappers.scikit_learn import KerasRegressor
from keras.callbacks import TensorBoard
from keras.optimizers import Adam, SGD
from keras.regularizers import l1, l2
from keras.initializers import RandomUniform, RandomNormal

from tensorflow.nn import relu, softmax

from sklearn import preprocessing
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from sklearn.preprocessing import RobustScaler, MinMaxScaler, StandardScaler, Normalizer
from sklearn.impute import SimpleImputer
from sklearn.datasets import make_regression
from sklearn.metrics import mean_squared_error, r2_score

import scipy.stats as st

import seaborn as sns

import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
experiment = Experiment(api_key="CndJ3YmXyZcxmsV8EccJEuu9C",
                        project_name="general", workspace="paologalligit")

COMET INFO: Experiment is live on comet.ml https://www.comet.ml/paologalligit/general/75c31ce2290e4764bea3ec90b9488a8a



In [3]:
df = pd.read_csv('fifa19.csv')

In [4]:
df.drop(columns=['Unnamed: 0', 'ID', 'Photo', 'Flag', 'Club Logo', 'Real Face', 'Preferred Foot',
                 'Body Type', 'Jersey Number', 'Joined', 'Loaned From', 'Contract Valid Until'],inplace=True)

## Obiettivo: predire valore dei giocatori

Pre-processing: convertire value, wage e release clause da string a float

In [5]:
curs=["Release Clause", "Value", "Wage"]

for cur in curs:
    def curr2val(x):
        x = str(x).replace('€', '')
        if 'K' in x: x = float(str(x).replace('K', '')) * 1000
        else: x = float(str(x).replace('M', '')) * 1000000
        return x
    df[cur] = df[cur].apply(curr2val)
    

Individuare eventuali outlier nella colonna value

In [6]:
def detect_outlier(data, threshold = 3):
    outliers=[]
    mean = np.mean(data)
    std = np.std(data)
    
    for y in data:
        score= (y - mean) / std 
        if np.abs(score) > threshold:
            outliers.append(y)
    return outliers

In [7]:
min_out = min(detect_outlier(df['Value'], threshold = 2))

df = df[df['Value'] < min_out] 
df = df[df['Value'] > 0]

Conversione in interi per le altre label

In [8]:
cols=["LS", "ST", "RS", "LW", "LF", "CF", "RF", "RW","LAM", "CAM", "RAM", "LM", "LCM", "CM", "RCM", "RM", "LWB", "LDM","CDM", "RDM", "RWB", "LB", "LCB", "CB", "RCB", "RB"]
for col in cols:
    df[col] = df[col].str[:-2]
    df[col] = df[col].astype(float)

In [9]:
df['Height'] = df['Height'].str.replace("'",'.')
df['Height'] = df['Height'].astype(float)

df['Weight'] = df['Weight'].str[:-3]
df['Weight'] = df['Weight'].astype(float)

Calcolo correlazione tra i valori per scegliere colonne significative

In [10]:
df_corr = df.corr()

# fig = plt.figure(figsize=(50,20))
# ax = fig.add_subplot(111)
# cax = ax.matshow(df_corr,cmap='coolwarm', vmin=-1, vmax=1)
# fig.colorbar(cax)

# ticks = np.arange(0,len(df_corr.columns),1)
# ax.set_xticks(ticks)
# ax.set_xticklabels(df_corr.columns)
# plt.xticks(rotation=90)
# ax.set_yticks(ticks)
# ax.set_yticklabels(df_corr.columns)

# plt.show()

In [11]:
labels = []
for label in df_corr:
#     if df_corr['Value'][label] < 0 or df_corr['Value'][label] > 0.5: labels.append(label)
    if df_corr['Value'][label] > 0.55: labels.append(label)
        
df_flt = df[labels]        
df_flt.head()      

,Overall,Potential,Value,Wage,LCM,CM,RCM,Reactions,Release Clause
41,88,88,4000000.0,77000.0,NaN,NaN,NaN,79.0,7400000.0
102,85,85,9000000.0,38000.0,70.0,70.0,70.0,85.0,15300000.0
108,85,85,9000000.0,57000.0,63.0,63.0,63.0,83.0,17100000.0
152,84,84,4200000.0,95000.0,63.0,63.0,63.0,80.0,6900000.0
201,83,83,13000000.0,70000.0,NaN,NaN,NaN,78.0,24700000.0


Mescolo le righe del dataset

In [12]:
df_flt = df_flt.sample(frac=1)

train_slice = int(len(df_flt) * 0.8)

train = df_flt[:train_slice]
test = df_flt[train_slice:]

In [13]:
y_train = train.loc[:, ['Value']]
X_train = train.drop(columns='Value')

y_test = test.loc[:, ['Value']]
X_test = test.drop(columns='Value')

Sostiuisco eventuali valori nan con la media della colonna

In [14]:
imputer = SimpleImputer(missing_values = np.nan, strategy = 'mean')
imputer = imputer.fit(X_train)
X_full = imputer.transform(X_train)

imputer = SimpleImputer(missing_values = np.nan, strategy = 'mean')
imputer = imputer.fit(y_train)
y_full = imputer.transform(y_train)

imputer = SimpleImputer(missing_values = np.nan, strategy = 'mean')
imputer = imputer.fit(X_test)
X_test_full = imputer.transform(X_test)

imputer = SimpleImputer(missing_values = np.nan, strategy = 'mean')
imputer = imputer.fit(y_test)
y_test_full = imputer.transform(y_test)

Scalo i valori, sia per i caratteri che per il target

In [15]:
# scaler = RobustScaler()
# scaler = scaler.fit(X_train)
# X_train = scaler.transform(X_train)

# X_train_scaled = preprocessing.scale(X_train)
scaler = StandardScaler().fit(X_full)
X_scaled = scaler.transform(X_full)

scaler_train = StandardScaler().fit(y_full)
y_scaled = scaler_train.transform(y_full) 
# X_train_scaled, X_test_scaled

scaler = StandardScaler().fit(X_test_full)
X_test_scaled = scaler.transform(X_test_full)

In [16]:
def coeff_determination(y_test, y_pred):
    from keras import backend as K
    SS_res =  K.sum(K.square( y_test-y_pred ))
    SS_tot = K.sum(K.square( y_test - K.mean(y_test) ) )
    return ( 1 - SS_res/(SS_tot + K.epsilon()))

In [17]:
def build_nn(neurons):
    model = Sequential()
    # Adding the input layer
    model.add(Dense(256, input_dim = neurons, activation='relu', kernel_regularizer=l2(0.001), kernel_initializer=RandomNormal(mean=0.0, stddev=0.05, seed=None)))
#     model.add(Dropout(0.1))
    
    # Adding the first hidden layer
    model.add(Dense(128, activation = 'relu', kernel_regularizer=l2(0.001), kernel_initializer=RandomNormal(mean=0.0, stddev=0.05, seed=None)))
#     model.add(Dropout(0.25))
    
    # Adding the second hidden layer
    model.add(Dense(64, activation = 'relu', kernel_regularizer=l2(0.001), kernel_initializer=RandomNormal(mean=0.0, stddev=0.05, seed=None)))
    
    # Adding the output layer
    model.add(Dense(1, activation = 'linear'))
    
    return model

In [18]:
ts_board = TensorBoard(log_dir='value_predictions_v3/{}'.format('kfold_' + str(time())))

In [19]:
def mean_abs_error(prediction, target):
    res = 0
    tot = 0
    for i in range(len(target)):
        if target[i][0] != 0 and not np.isnan(prediction[i][0]):
#             print(prediction[i][0], target[i][0])
            res += abs(prediction[i][0] - target[i][0])
            tot += 1
    print(tot, len(target), len(prediction))
    return round(res / tot, 2)

# Cross-validation
### K-Fold

In [20]:
# kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
kfold = KFold(n_splits=10, random_state=42, shuffle=False)

In [21]:
scores = []
abs_errors = []

nn = build_nn(X_scaled.shape[1])

opt = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
# opt = SGD(lr=0.01, momentum=0.9)
nn.compile(optimizer = opt, loss = 'mean_absolute_error', metrics = [coeff_determination])

Instructions for updating:
Colocations handled automatically by placer.


In [22]:
%%time

stage = 0

for train, test in kfold.split(X_scaled):
    
    stage += 1
    
    print('#'*60)
    print('\nSTAGE --> {}\n'.format(stage))
    print('#'*60)
    
    X_train_round, X_test_round, y_train_round, y_test_round = X_scaled[train], X_scaled[test], y_scaled[train], y_scaled[test]
        
    nn.fit(X_train_round, y_train_round, batch_size=125, epochs=100, shuffle=True)
    
    # predict
    res = nn.predict(X_test_round)
    print('R2 of round: ', round(r2_score(y_test_round, res), 4))
    
    # evaluate
    loss, acc = nn.evaluate(X_train_round, y_train_round, verbose=1)
    
    reversed_res = scaler_train.inverse_transform(res)
    reversed_label = scaler_train.inverse_transform(y_test_round)
    abs_errors.append(
        {'ACC': acc * 100, 'MAE': mean_abs_error(reversed_res, reversed_label), 'MAX': max(reversed_label), 'MEAN': round(np.mean(reversed_label), 4)}
    )
    
    print("loss: %.4f, acc: %.4f%%" % (loss, acc*100))
    if acc < 0:
        print('X_test round:\n')
        print(X_test_round)
        print('#'*60)
        print('y_test_round:\n')
        print(y_test_round)
        print('#'*60)
    
    scores.append(acc * 100)

############################################################

STAGE --> 1

############################################################
Instructions for updating:
Use tf.cast instead.
Epoch 1/500
12535/12535 [==============================] - 1s 80us/step - loss: 0.2484 - coeff_determination: 0.8511
Epoch 2/500
12535/12535 [==============================] - 0s 20us/step - loss: 0.1432 - coeff_determination: 0.9393
Epoch 3/500
12535/12535 [==============================] - 0s 21us/step - loss: 0.1261 - coeff_determination: 0.9412
Epoch 4/500
12535/12535 [==============================] - 0s 20us/step - loss: 0.1179 - coeff_determination: 0.9411
Epoch 5/500
12535/12535 [==============================] - 0s 20us/step - loss: 0.1122 - coeff_determination: 0.9428
Epoch 6/500
12535/12535 [==============================] - 0s 21us/step - loss: 0.1066 - coeff_determination: 0.9485
Epoch 7/500
12535/12535 [==============================] - 0s 21us/step - loss: 0.1031 - coeff_determination: 0.94

12535/12535 [==============================] - 0s 20us/step - loss: 0.0712 - coeff_determination: 0.9765
Epoch 68/500
12535/12535 [==============================] - 0s 20us/step - loss: 0.0716 - coeff_determination: 0.9760
Epoch 69/500
12535/12535 [==============================] - 0s 20us/step - loss: 0.0720 - coeff_determination: 0.9768
Epoch 70/500
12535/12535 [==============================] - 0s 20us/step - loss: 0.0717 - coeff_determination: 0.9759
Epoch 71/500
12535/12535 [==============================] - 0s 19us/step - loss: 0.0702 - coeff_determination: 0.9763
Epoch 72/500
12535/12535 [==============================] - 0s 20us/step - loss: 0.0710 - coeff_determination: 0.9778
Epoch 73/500
12535/12535 [==============================] - 0s 20us/step - loss: 0.0710 - coeff_determination: 0.9768
Epoch 74/500
12535/12535 [==============================] - 0s 20us/step - loss: 0.0712 - coeff_determination: 0.9761
Epoch 75/500
12535/12535 [==============================] - 0s 20us/s

12535/12535 [==============================] - 0s 19us/step - loss: 0.0700 - coeff_determination: 0.9778
Epoch 137/500
12535/12535 [==============================] - 0s 19us/step - loss: 0.0673 - coeff_determination: 0.9793
Epoch 138/500
12535/12535 [==============================] - 0s 20us/step - loss: 0.0670 - coeff_determination: 0.9793
Epoch 139/500
12535/12535 [==============================] - 0s 19us/step - loss: 0.0697 - coeff_determination: 0.9785
Epoch 140/500
12535/12535 [==============================] - 0s 19us/step - loss: 0.0672 - coeff_determination: 0.9800
Epoch 141/500
12535/12535 [==============================] - 0s 19us/step - loss: 0.0669 - coeff_determination: 0.9798
Epoch 142/500
12535/12535 [==============================] - 0s 19us/step - loss: 0.0685 - coeff_determination: 0.9795
Epoch 143/500
12535/12535 [==============================] - 0s 19us/step - loss: 0.0674 - coeff_determination: 0.9797
Epoch 144/500
12535/12535 [==============================] - 0

Epoch 205/500
12535/12535 [==============================] - 0s 20us/step - loss: 0.0649 - coeff_determination: 0.9814
Epoch 206/500
12535/12535 [==============================] - 0s 20us/step - loss: 0.0671 - coeff_determination: 0.9807
Epoch 207/500
12535/12535 [==============================] - 0s 19us/step - loss: 0.0654 - coeff_determination: 0.9816
Epoch 208/500
12535/12535 [==============================] - 0s 20us/step - loss: 0.0657 - coeff_determination: 0.9816
Epoch 209/500
12535/12535 [==============================] - 0s 20us/step - loss: 0.0660 - coeff_determination: 0.9811
Epoch 210/500
12535/12535 [==============================] - 0s 20us/step - loss: 0.0663 - coeff_determination: 0.9804
Epoch 211/500
12535/12535 [==============================] - 0s 20us/step - loss: 0.0656 - coeff_determination: 0.9814
Epoch 212/500
12535/12535 [==============================] - 0s 19us/step - loss: 0.0653 - coeff_determination: 0.9807
Epoch 213/500
12535/12535 [=====================

12535/12535 [==============================] - 0s 19us/step - loss: 0.0643 - coeff_determination: 0.9825
Epoch 274/500
12535/12535 [==============================] - 0s 19us/step - loss: 0.0643 - coeff_determination: 0.9827
Epoch 275/500
12535/12535 [==============================] - 0s 19us/step - loss: 0.0644 - coeff_determination: 0.9822
Epoch 276/500
12535/12535 [==============================] - 0s 19us/step - loss: 0.0642 - coeff_determination: 0.9819
Epoch 277/500
12535/12535 [==============================] - 0s 19us/step - loss: 0.0653 - coeff_determination: 0.9828
Epoch 278/500
12535/12535 [==============================] - 0s 19us/step - loss: 0.0643 - coeff_determination: 0.9826
Epoch 279/500
12535/12535 [==============================] - 0s 19us/step - loss: 0.0645 - coeff_determination: 0.9825
Epoch 280/500
12535/12535 [==============================] - 0s 19us/step - loss: 0.0671 - coeff_determination: 0.9819
Epoch 281/500
12535/12535 [==============================] - 0

...

Epoch 219/500
12536/12536 [==============================] - 0s 21us/step - loss: 0.0561 - coeff_determination: 0.9880
Epoch 220/500
12536/12536 [==============================] - 0s 19us/step - loss: 0.0561 - coeff_determination: 0.9876
Epoch 221/500
12536/12536 [==============================] - 0s 18us/step - loss: 0.0569 - coeff_determination: 0.9874
Epoch 222/500
12536/12536 [==============================] - 0s 18us/step - loss: 0.0562 - coeff_determination: 0.9871
Epoch 223/500
12536/12536 [==============================] - 0s 19us/step - loss: 0.0579 - coeff_determination: 0.9872
Epoch 224/500
12536/12536 [==============================] - 0s 21us/step - loss: 0.0563 - coeff_determination: 0.9877
Epoch 225/500
12536/12536 [==============================] - 0s 20us/step - loss: 0.0563 - coeff_determination: 0.9875
Epoch 226/500
12536/12536 [==============================] - 0s 19us/step - loss: 0.0562 - coeff_determination: 0.9875
Epoch 227/500
12536/12536 [=====================

12536/12536 [==============================] - 0s 20us/step - loss: 0.0570 - coeff_determination: 0.9872
Epoch 288/500
12536/12536 [==============================] - 0s 19us/step - loss: 0.0571 - coeff_determination: 0.9874
Epoch 289/500
12536/12536 [==============================] - 0s 19us/step - loss: 0.0564 - coeff_determination: 0.9879
Epoch 290/500
12536/12536 [==============================] - 0s 19us/step - loss: 0.0560 - coeff_determination: 0.9878
Epoch 291/500
12536/12536 [==============================] - 0s 19us/step - loss: 0.0573 - coeff_determination: 0.9862
Epoch 292/500
12536/12536 [==============================] - 0s 19us/step - loss: 0.0568 - coeff_determination: 0.9880
Epoch 293/500
12536/12536 [==============================] - 0s 19us/step - loss: 0.0567 - coeff_determination: 0.9871
Epoch 294/500
12536/12536 [==============================] - 0s 19us/step - loss: 0.0575 - coeff_determination: 0.9872
Epoch 295/500
12536/12536 [==============================] - 0

Epoch 356/500
12536/12536 [==============================] - 0s 18us/step - loss: 0.0561 - coeff_determination: 0.9878
Epoch 357/500
12536/12536 [==============================] - 0s 21us/step - loss: 0.0565 - coeff_determination: 0.9873
Epoch 358/500
12536/12536 [==============================] - 0s 19us/step - loss: 0.0568 - coeff_determination: 0.9875
Epoch 359/500
12536/12536 [==============================] - 0s 19us/step - loss: 0.0558 - coeff_determination: 0.9883
Epoch 360/500
12536/12536 [==============================] - 0s 19us/step - loss: 0.0563 - coeff_determination: 0.9876
Epoch 361/500
12536/12536 [==============================] - 0s 19us/step - loss: 0.0561 - coeff_determination: 0.9877
Epoch 362/500
12536/12536 [==============================] - 0s 19us/step - loss: 0.0554 - coeff_determination: 0.9878
Epoch 363/500
12536/12536 [==============================] - 0s 19us/step - loss: 0.0564 - coeff_determination: 0.9871
Epoch 364/500
12536/12536 [=====================

12536/12536 [==============================] - 0s 20us/step - loss: 0.0579 - coeff_determination: 0.9867
Epoch 425/500
12536/12536 [==============================] - 0s 20us/step - loss: 0.0566 - coeff_determination: 0.9875
Epoch 426/500
12536/12536 [==============================] - 0s 20us/step - loss: 0.0558 - coeff_determination: 0.9882
Epoch 427/500
12536/12536 [==============================] - 0s 20us/step - loss: 0.0562 - coeff_determination: 0.9879
Epoch 428/500
12536/12536 [==============================] - 0s 20us/step - loss: 0.0577 - coeff_determination: 0.9866
Epoch 429/500
12536/12536 [==============================] - 0s 19us/step - loss: 0.0581 - coeff_determination: 0.9868
Epoch 430/500
12536/12536 [==============================] - 0s 20us/step - loss: 0.0585 - coeff_determination: 0.9867
Epoch 431/500
12536/12536 [==============================] - 0s 19us/step - loss: 0.0568 - coeff_determination: 0.9875
Epoch 432/500
12536/12536 [==============================] - 0

Epoch 493/500
12536/12536 [==============================] - 0s 20us/step - loss: 0.0562 - coeff_determination: 0.9879
Epoch 494/500
12536/12536 [==============================] - 0s 20us/step - loss: 0.0565 - coeff_determination: 0.9875
Epoch 495/500
12536/12536 [==============================] - 0s 19us/step - loss: 0.0560 - coeff_determination: 0.9880
Epoch 496/500
12536/12536 [==============================] - 0s 20us/step - loss: 0.0566 - coeff_determination: 0.9876
Epoch 497/500
12536/12536 [==============================] - 0s 19us/step - loss: 0.0568 - coeff_determination: 0.9867
Epoch 498/500
12536/12536 [==============================] - 0s 19us/step - loss: 0.0562 - coeff_determination: 0.9880
Epoch 499/500
12536/12536 [==============================] - 0s 19us/step - loss: 0.0561 - coeff_determination: 0.9881
Epoch 500/500
12536/12536 [==============================] - 0s 19us/step - loss: 0.0563 - coeff_determination: 0.9881
R2 of round:  0.982
12536/12536 [===============

In [23]:
print("%.2f%% (+/- %.2f%%)" % (np.mean(scores), np.std(scores) * 100))
abs_errors

98.51% (+/- 13.22%)


[{'ACC': 98.31013162296799,
  'MAE': 151633.87,
  'MAX': array([13500000.]),
  'MEAN': 1739770.28},
 {'ACC': 98.52608261519235,
  'MAE': 139615.62,
  'MAX': array([13500000.]),
  'MEAN': 1661837.7602},
 {'ACC': 98.44936355633426,
  'MAE': 138498.43,
  'MAX': array([13000000.]),
  'MEAN': 1730495.3338},
 {'ACC': 98.45422170387592,
  'MAE': 127217.52,
  'MAX': array([13000000.]),
  'MEAN': 1636848.5284},
 {'ACC': 98.33796020898869,
  'MAE': 120224.24,
  'MAX': array([13500000.]),
  'MEAN': 1619547.7387},
 {'ACC': 98.39885734399097,
  'MAE': 138992.77,
  'MAX': array([13500000.]),
  'MEAN': 1659145.7286},
 {'ACC': 98.65775205340573,
  'MAE': 127869.52,
  'MAX': array([13500000.]),
  'MEAN': 1649411.3424},
 {'ACC': 98.61926670563282,
  'MAE': 117570.64,
  'MAX': array([13500000.]),
  'MEAN': 1587067.4803},
 {'ACC': 98.70714612956554,
  'MAE': 139459.09,
  'MAX': array([13500000.]),
  'MEAN': 1767205.4598},
 {'ACC': 98.62479998872169,
  'MAE': 138916.76,
  'MAX': array([13500000.]),
  'MEAN

In [24]:
res = nn.predict(X_test_scaled)

# y_test_wrong = y_scaled[:3483]

scaler_test = StandardScaler().fit(y_test.values)
res = scaler_test.inverse_transform(res)

mean_abs_error(res, y_test.values), max(y_test.values), round(np.mean(y_test.values), 2), round(r2_score(y_test.values, res), 4)

# scaler_test = StandardScaler().fit(y_test)
# res = scaler_test.inverse_transform(res)
# y_test_wrong = scaler_test.inverse_transform(y_test_wrong)
# print(res)

# mean_abs_error(res, y_test_wrong), max(y_test_wrong), round(np.mean(y_test_wrong), 2), round(r2_score(y_test_wrong, res), 4)

3483 3483 3483


(138895.65, array([13500000.]), 1732052.83, 0.9844)